In [2]:
import pandas as pd
from pandas import read_csv
from datetime import datetime
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import LSTM
from numpy import concatenate
from math import sqrt
from pylab import mpl


In [3]:
# Read the data
df = pd.read_csv('file.csv',
                 parse_dates={'dt' : ['year', 'month', 'day','hour']},
                 #sep=";", 
                 infer_datetime_format=True,
                 low_memory=False, na_values=['nan','?'], index_col='dt')

df.drop('No', axis=1, inplace=True)
print(df.head(5))
new_index = pd.date_range(start='2010-01-01 00:00:00', periods=len(df), freq='H')
df.set_index(new_index, inplace=True)
df.index.name = 'dt'
print(df.head())
dataset_train_actual = df.copy()
dataset_train = df.copy()
print(dataset_train.shape)
print(dataset_train.head(5))

            PM2.5   PM10    CO    SO2   NO2     O3  Unnamed: 11  Unnamed: 12
dt                                                                          
2010 1 1 0  20.21  23.56  0.29   8.65  4.58  63.44          NaN            1
2010 1 1 1  20.41  23.96  0.29   9.28  5.24  62.53          NaN            2
2010 1 1 2  21.17  24.78  0.30  10.07  5.47  62.24          NaN            3
2010 1 1 3  22.24  26.04  0.30  10.36  5.28  63.58          NaN            4
2010 1 1 4  19.68  22.34  0.29   7.62  3.58  66.21          NaN            5
                     PM2.5   PM10    CO    SO2   NO2     O3  Unnamed: 11  \
dt                                                                         
2010-01-01 00:00:00  20.21  23.56  0.29   8.65  4.58  63.44          NaN   
2010-01-01 01:00:00  20.41  23.96  0.29   9.28  5.24  62.53          NaN   
2010-01-01 02:00:00  21.17  24.78  0.30  10.07  5.47  62.24          NaN   
2010-01-01 03:00:00  22.24  26.04  0.30  10.36  5.28  63.58          NaN   
2010-

C:\Users\Admin\AppData\Local\Temp\ipykernel_41332\792585478.py:2: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv('第二次 - 现场+模型-原始.csv',
C:\Users\Admin\AppData\Local\Temp\ipykernel_41332\792585478.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv('第二次 - 现场+模型-原始.csv',
C:\Users\Admin\AppData\Local\Temp\ipykernel_41332\792585478.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('第二次 - 现场+模型-原始.csv',
C:\Users\Admin\AppData\Local\Temp\ipykernel_41332\792585478.py:11: 

In [5]:
import pandas as pd
import numpy as np
index = pd.date_range(start='2010-07-30 02:00:00', periods=10, freq='H')
random_data = pd.Series(np.random.rand(10), name='Random Data')
datelist_future= pd.DataFrame({'dt': index, 'Random Data': random_data})
print(datelist_future)

                   dt  Random Data
0 2010-07-30 02:00:00     0.195626
1 2010-07-30 03:00:00     0.304068
2 2010-07-30 04:00:00     0.209520
3 2010-07-30 05:00:00     0.808487
4 2010-07-30 06:00:00     0.718004
5 2010-07-30 07:00:00     0.153627
6 2010-07-30 08:00:00     0.915427
7 2010-07-30 09:00:00     0.553904
8 2010-07-30 10:00:00     0.482990
9 2010-07-30 11:00:00     0.621591


C:\Users\Admin\AppData\Local\Temp\ipykernel_41332\3077675557.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  index = pd.date_range(start='2010-07-30 02:00:00', periods=10, freq='H')


In [6]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
training_set_scaled = sc.fit_transform(training_set)
sc_predict = StandardScaler()
sc_predict.fit_transform(training_set[:, 0:1])

D:\anaconda3\envs\python310\lib\site-packages\sklearn\utils\extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
D:\anaconda3\envs\python310\lib\site-packages\sklearn\utils\extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
D:\anaconda3\envs\python310\lib\site-packages\sklearn\utils\extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


array([[-0.09569504],
       [-0.08925316],
       [-0.06477404],
       ...,
       [-0.74213703],
       [-0.74181494],
       [-0.74117075]])

In [7]:
X_train = []
y_train = []
n_future = 3 
n_past = 30 
for i in range(n_past, len(training_set_scaled) - n_future +1):
    X_train.append(training_set_scaled[i - n_past:i, [0, 1, 3,4,5]])
    y_train.append(training_set_scaled[i+n_future-1:i+n_future, 2])
X_train, y_train = np.array(X_train), np.array(y_train)

print('X_train shape == {}.'.format(X_train.shape))
print('y_train shape == {}.'.format(y_train.shape))


X_train shape == (42217, 30, 5).
y_train shape == (42217, 1).


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
print( X_train.shape)
print( X_test.shape)
print(y_train.shape)
print( y_test.shape)


X_train_concentration = X_train[ :, :, :4]  
X_train_angle = X_train[ :, :, 4:5]          
X_train_wind_speed = X_train[ :, :, 5:6]     
X_train_distance = X_train[:, :, 6:7]
print(X_train_concentration.shape)
print(X_train_angle.shape)
print(X_train_wind_speed.shape)
print(X_train_distance.shape)

训练集特征变量形状: (33773, 30, 5)
测试集特征变量形状: (8444, 30, 5)
训练集目标变量形状: (33773, 1)
测试集目标变量形状: (8444, 1)
(33773, 30, 4)
(33773, 30, 1)
(33773, 30, 0)
(33773, 30, 0)


In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv2D, LSTM, Dense, Bidirectional, Dropout
from keras.layers import Reshape,Permute
from tensorflow.keras.layers import Lambda

def lr_schedule(epoch):
    initial_learning_rate = 1e-3
    decay_rate = 0.5
    decay_steps = 10
    if epoch < decay_steps:
        return initial_learning_rate
    else:
        return initial_learning_rate * decay_rate ** ((epoch - decay_steps) // decay_steps)


lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

input_shape = X_train.shape[1:]
inputs = keras.Input(shape=input_shape)

print(inputs.shape)

#Specific content replacement of different models

def correlation_coefficient(y_true, y_pred):
    
    mean_pred = tf.reduce_mean(y_pred)
    mean_true = tf.reduce_mean(y_true)
    xm = y_pred - mean_pred
    ym = y_true - mean_true
    r_num = tf.reduce_sum(tf.multiply(xm, ym))
    r_den = tf.sqrt(tf.reduce_sum(tf.square(xm)) * tf.reduce_sum(tf.square(ym)))
    r = r_num / r_den
    return r
def total_inertia_criterion(y_true, y_pred):
    
    tss = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    rss = tf.reduce_sum(tf.square(y_true - y_pred))
    tic = 1 - (rss / tss)
    return tic
def index_of_agreement(y_true, y_pred):
    
    mean_true = tf.reduce_mean(y_true)
    ia_numerator = tf.reduce_sum(tf.square(tf.subtract(y_true, y_pred)))
    ia_denominator = tf.reduce_sum(tf.square(tf.abs(tf.subtract(y_true, mean_true)))) + tf.reduce_sum(tf.square(tf.abs(tf.subtract(y_pred, mean_true))))
    ia = 1 - (ia_numerator / ia_denominator)
    return ia

model = tf.keras.Model(inputs=inputs, outputs=output)

model.compile(optimizer='adam', loss='mse', metrics=["msle", "mae", "mape", "RootMeanSquaredError", correlation_coefficient, total_inertia_criterion, index_of_agreement])

model.summary()

inputs形状: (None, 30, 5)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 30, 5)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 30, 64)              │             384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 30, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 30, 64)              │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 1920)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │           1,921 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,465 (25.25 KB)

 Trainable params: 6,465 (25.25 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:

history = model.fit(X_train, y_train, epochs=150, validation_data=(X_test, y_test), callbacks=[lr_scheduler])
import pandas as pd


history_dict = history.history

history_df = pd.DataFrame(history_dict)

# 将 DataFrame 保存为 CSV 文件
history_df.to_csv(results.csv', index=False)

Epoch 1/150
1056/1056 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - RootMeanSquaredError: 0.6739 - correlation_coefficient: 0.7765 - index_of_agreement: 0.7188 - loss: 0.4593 - mae: 0.3516 - mape: 192.4875 - msle: 0.0617 - total_inertia_criterion: 0.5410 - val_RootMeanSquaredError: 0.5574 - val_correlation_coefficient: 0.8648 - val_index_of_agreement: 0.8233 - val_loss: 0.3107 - val_mae: 0.2923 - val_mape: 174.1651 - val_msle: 0.0347 - val_total_inertia_criterion: 0.6895 - learning_rate: 0.0010
Epoch 2/150
1056/1056 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - RootMeanSquaredError: 0.5393 - correlation_coefficient: 0.8719 - index_of_agreement: 0.8401 - loss: 0.2912 - mae: 0.2778 - mape: 198.1050 - msle: 0.0370 - total_inertia_criterion: 0.7056 - val_RootMeanSquaredError: 0.5091 - val_correlation_coefficient: 0.8894 - val_index_of_agreement: 0.8655 - val_loss: 0.2591 - val_mae: 0.2290 - val_mape: 176.5759 - val_msle: 0.0300 - val_total_inertia_criterion: 0.7465 - learning_rate: 0.0010
Epoch 3/150
1056/1056 

KeyError: 'lr'

In [12]:
from sklearn.metrics import r2_score
train_predictions = model.predict(X_train)
train_r2 = r2_score(y_train, train_predictions)
print(train_r2)
test_predictions = model.predict(X_test)
test_r2 = r2_score(y_test, test_predictions)
print(test_r2)

1056/1056 ━━━━━━━━━━━━━━━━━━━━ 1s 622us/step
训练集的 R2 分数: 0.927938089982882
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 593us/step
测试集的 R2 分数: 0.9098366050377286
